# Deep Learning models 

In [21]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autotime
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import model_selection, preprocessing
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D, Conv1D, SimpleRNN
from keras.models import Model
from keras.models import Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers import Dense, Input, Flatten, Dropout, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, Embedding

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 4.92 ms


In [8]:

df= pd.read_csv("consumer_complaints.csv")
df1 = df.dropna(subset= ['consumer_complaint_narrative'])
df1['consumer_complaint_narrative']= df1['consumer_complaint_narrative'].str.lower()
df1['consumer_complaint_narrative'] =df1['consumer_complaint_narrative'].str.replace(r'[^\w\s]',"")
df1['consumer_complaint_narrative'] = df1['consumer_complaint_narrative'].str.replace(r"xx+\s","")
df1['consumer_complaint_narrative'].head(1)

190126    has claimed i owe them 2700 for years despite ...
Name: consumer_complaint_narrative, dtype: object

time: 4.25 s


In [9]:
train_x, valid_x, train_y, valid_y = train_test_split(df1['consumer_complaint_narrative'], df1['product'],stratify=df1['product'], 
                                                    test_size=0.25)


time: 105 ms


Tokenizer This class allows to vectorize a text corpus, by turning each text into either a sequence of integers (each integer being the index of a token in a dictionary) or into a vector where the coefficient for each token could be binary, based on word count, based on tf-idf...


In [10]:
tokenizer = Tokenizer(num_words=25000)

time: 1.02 ms


Updates internal vocabulary based on a list of texts. In the case where texts contains lists, we assume each entry of the lists to be a token. Required before using texts_to_sequences or texts_to_matrix.


In [11]:
tokenizer.fit_on_texts(train_x.values)

time: 7.86 s


In [12]:
train_sequences = tokenizer.texts_to_sequences(train_x.values)
test_sequences = tokenizer.texts_to_sequences(valid_x.values)

time: 7.15 s


Padding pad_sequences is used to ensure that all sequences in a list have the same length. By default this is done by padding 0 in the beginning of each sequence until each sequence has the same length as the longest sequence. padding= 'post' will append 0 at the end of the sequence

In [13]:
print('Found %s unique tokens.' % len(set(tokenizer.word_index)))

Found 57761 unique tokens.
time: 12 ms


In [16]:
total_complaints = np.append(train_x.values,valid_x.values)
MAX_SEQUENCE_LENGTH = max([len(c.split()) for c in total_complaints])
MAX_SEQUENCE_LENGTH

807

time: 701 ms


In [17]:
train_data = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH,padding='post')
test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH,padding='post')

time: 1.53 s


In [18]:
print(train_data.shape)
print(test_data.shape)

(50104, 807)
(16702, 807)
time: 1.35 ms


In [22]:
enc = preprocessing.LabelEncoder()
train_labels = enc.fit_transform(train_y)
test_labels = enc.fit_transform(valid_y)

time: 17.9 ms


In [23]:
print(enc.classes_)
print(np.unique(train_labels, return_counts=True))
print(np.unique(test_labels, return_counts=True))

['Bank account or service' 'Consumer Loan' 'Credit card'
 'Credit reporting' 'Debt collection' 'Money transfers' 'Mortgage'
 'Other financial service' 'Payday loan' 'Prepaid card' 'Student loan']
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]), array([ 4283,  2758,  5947,  9394, 13164,   500, 11189,    83,   544,
         646,  1596]))
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]), array([1428,  920, 1982, 3132, 4388,  166, 3730,   27,  182,  215,  532]))
time: 7.74 ms


to_categorical behave like onehotencoding technique if your particular categories is present then it mark as 1 else 0 in remain row